In [ ]:
import re
import pandas
import numpy
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing import sequence

In [ ]:
DEFAULT_PATH ='~/.kaggle/competitions/word2vec-nlp-tutorial/' # 파일이 저장된 경로
 
# 데이터 프레임 형태로 학습 데이터를 불러온다.
# header=0 : 파일의 첫 번째에 열 이름이 표시된다.
# delimiter="\t" :  tab을 기준으로 구분한다
# quoting=3 : 쌍따옴표를 무시한다.
train = pd.read_csv(DEFAULT_PATH+"labeledTrainData.tsv", header=0, delimiter="\t", quoting=3) 

In [ ]:
def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()
    
    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)
        
    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [ ]:
#train데이터를 전처리
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

MAX_SEQUENCE_LENGTH = 50 # 문장 최대 길이

inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
labels = np.array(train['sentiment']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', inputs.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', labels.shape) # 감정 데이터 형태 확인

In [ ]:
# 경로 및 파일 이름 지정
FILE_DIR_PATH = './data/'
INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

data_configs = word_vocab
data_configs['vocab_size'] = len(word_vocab) # vocab size 추가

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(FILE_DIR_PATH):
	os.makedirs(FILE_DIR_PATH)
# 전처리 된 데이터를 넘파이 형태로 저장
np.save(open(FILE_DIR_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'wb'), inputs)
np.save(open(FILE_DIR_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'wb'), labels)

# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(FILE_DIR_PATH + DATA_CONFIGS_FILE_NAME, 'w'), ensure_ascii=False)

In [ ]:
test = pd.read_csv(DEFAULT_PATH+"testData.tsv", header=0, delimiter="\t", quoting=3)

clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append(preprocessing(review, remove_stopwords = True))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_test_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_test_reviews)

MAX_SEQUENCE_LENGTH = 50

test_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

INPUT_TEST_DATA_FILE_NAME = 'test_input.npy'

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(FILE_DIR_PATH):
    os.makedirs(FILE_DIR_PATH)

np.save(open(FILE_DIR_PATH + INPUT_TEST_DATA_FILE_NAME, 'wb'), test_inputs)